In [ ]:
pip install requests beautifulsoup4 pandas

In [ ]:
import pandas as pd
data1 = pd.read_excel("/content/proj1_dataset.xlsx")

In [ ]:
data1.head()

,Holiday,Source,Year,Author,Link
0,MLK,FOX,2024,Greg Wehner,https://www.foxnews.com/politics/fbis-post-hon...
1,MLK,FOX,2023,Aubrie Spady,https://www.foxnews.com/politics/president-bid...
2,MLK,FOX,2022,Lisa Bennatan,https://www.foxnews.com/us/mlk-march-frederick...
3,MLK,FOX,2021,Paul Steinhauser,https://www.foxnews.com/politics/biden-mlk-day...
4,MLK,FOX,2020,Andrew O'Reilly,https://www.foxnews.com/politics/trump-martin-...


In [ ]:
import requests
from bs4 import BeautifulSoup

urls = data1['Link']

header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"
}

# Function to scrape article content from a URL
def scrape_article(url):
    try:
        # Send a GET request to the URL
        if "wsj.com" in url:
          response = requests.get(url, headers=header)
        else:
          response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful

        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the headline
        headline = soup.find('h1').get_text(strip=True)

        # Find the article content
        if "apnews" in url:
            article_content = soup.find('div', class_='RichTextStoryBody')
        elif "cnn" in url:
            article_content = soup.find('div', class_='article__content')
        else: # "foxnews" in url
            article_content = soup.find('div', class_='article-body')

        paragraphs = article_content.find_all('p')
        content = ' '.join([para.get_text(strip=True) for para in paragraphs])

        return {
            'URL': url,
            'Headline': headline,
            'Content': content
        }

    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# List to store the scraped data
scraped_data = []

# Loop through the list of URLs and scrape each article
for url in urls:
    article_data = scrape_article(url)
    if article_data:
        scraped_data.append(article_data)

# Convert the list of scraped data into a DataFrame
df = pd.DataFrame(scraped_data)

# save the DataFrame to a CSV file
df.to_csv('scraped_articles_test.csv', index=False)

print(df.head())


Error scraping https://www.wsj.com/lifestyle/travel/martin-luther-king-jr-atlanta-neighborhood-walking-tour-3518cfc9: 401 Client Error: HTTP Forbidden for url: https://www.wsj.com/lifestyle/travel/martin-luther-king-jr-atlanta-neighborhood-walking-tour-3518cfc9
Error scraping https://www.wsj.com/articles/martin-luther-king-would-choose-reflection-over-intersectionality-discrimination-gender-race-sexual-orientation-11673645591: 401 Client Error: HTTP Forbidden for url: https://www.wsj.com/articles/martin-luther-king-would-choose-reflection-over-intersectionality-discrimination-gender-race-sexual-orientation-11673645591
Error scraping https://www.wsj.com/articles/family-of-martin-luther-king-jr-to-lead-voting-rights-march-in-washington-11642107094: 401 Client Error: HTTP Forbidden for url: https://www.wsj.com/articles/family-of-martin-luther-king-jr-to-lead-voting-rights-march-in-washington-11642107094
Error scraping https://www.wsj.com/articles/how-the-left-hijacked-civil-rights-1161074

In [ ]:
len(df)

85

In [ ]:
merged_df = pd.merge(data1, df, left_on='Link', right_on='URL', how='inner')

In [ ]:
merged_df.to_csv('scraped_articles.csv', index=False)